<a href="https://colab.research.google.com/github/Gajendra-theDataEngineer004627/DataEngineeringProjects/blob/ChessChampionshipProject/DE_Solutions_GRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -y

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,347 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [32.0 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,372 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/repos

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
!pip install -q findspark
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("<app_name>")
 .getOrCreate())


In [ ]:
# function to doanload data files from the google drive
import gdown
def downloadfiles_fromFolder() :
    url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"   # can also be used input()
    if url.split("/")[-1]== "?usp=sharing":
       url = url.replace("?usp=sharing","")
    gdown.download_folder(url, output="/content")

downloadfiles_fromFolder()

In [9]:
# Creating the datasets for each data files and understanding the data

chess_wc_history_game_info = spark.read.load("/content/chess_wc_history_game_info.csv",format="csv", sep=",", inferSchema="true", header="true")

chess_wc_history_moves = spark.read.load("/content/chess_wc_history_moves.csv",format="csv", sep=",", inferSchema="true", header="true")

eco_codes = spark.read.load("/content/eco_codes.csv",format="csv", sep=",", inferSchema="true", header="true")


In [ ]:
# checking the top 10 records are properly loaded or not
# print(chess_wc_history_moves.head())
# print(eco_codes.head())
# print(chess_wc_history_game_info.head())

In [13]:
import os
import pandas as pd

def spark_dataframes_to_csv(dataframes, drive_path):
    # Mount Google Drive if not already mounted
    if not os.path.isdir('/content/gdrive'):
        from google.colab import drive
        drive.mount('/content/gdrive')

    # Create directory if it doesn't exist
    directory_path = '/content/gdrive/MyDrive/' + drive_path
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    # Convert Spark DataFrames to Pandas DataFrames and save as CSV
    for i, dataframe in enumerate(dataframes, start=1):
        pandas_df = dataframe.toPandas()
        file_path = f'{directory_path}/df{i}.csv'
        pandas_df.to_csv(file_path, index=False)
        print(f"DataFrame {i} saved as {file_path}")



In [15]:
# Creating temporary view for data manupulations
chess_wc_history_moves.createOrReplaceTempView("chess_wc_history_movesTemp")
eco_codes.createOrReplaceTempView("eco_codesTemp")
chess_wc_history_game_info.createOrReplaceTempView("chess_wc_history_game_infoTemp")

# ----Creating the table from the temp view so that the tables can be accessible globally(accessed across different Spark sessions) #

query_main_Event  = """Select * from chess_wc_history_game_infoTemp where
                        event not like '%KO%' and
                        event not like '%k.o.%' """


# -------------WCh knock out(string with k.o.,KO ) are not included in main event-----------#
spark.sql(query_main_Event).createOrReplaceTempView("chess_wc_history_game_infoTemp1")


chess_wc_history_movesTempTable = spark.table("chess_wc_history_movesTemp")
eco_codesTempTable = spark.table("eco_codesTemp")
chess_wc_history_game_infoTempTable = spark.table("chess_wc_history_game_infoTemp1")

# Checking the validity of data in table form  using mysql like capabilities in SparkSql
#result1 = spark.sql("SELECT * FROM chess_wc_history_movesTemp limit 5")
#result1.show()
#result2 = spark.sql("SELECT * FROM eco_codesTemp")
#result2.show()
# result2.printSchema()    # --> to check schema
  # result3 = spark.sql("SELECT * from chess_wc_history_game_infoTemp limit 5 ")
  # result3.show()
 #spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
  # eco_codes.columns    # this is used to check the data frames columns
  #chess_wc_history_game_info.columns


query1 = """ WITH CTE_WInner_List AS
                (select winner , event,round , tournament_name,date_played,
                count(*) over (partition by winner,tournament_name order by date_played) as WinnerCount
                from chess_wc_history_game_infoTemp1
                where winner not like '%draw%'and
                tournament_name like 'WorldChamp%' )
                SELECT winner, tournament_name, MAX(WinnerCount) AS Score
                FROM CTE_Winner_List
                GROUP BY winner ,tournament_name
                order BY tournament_name DESC
"""
spark.sql(query1).createOrReplaceTempView("temp_view1")

final_query1 = """ with CTE_Final AS (
                select winner, tournament_name, Score ,
                ROW_NUMBER() OVER (partition by tournament_name order by Score DESC) as Row_num
                from temp_view1 )
                select winner, tournament_name  --, Score ,Row_num
                from CTE_Final
                where Row_num = 1
                order by tournament_name DESC
"""

#  -----------Sollution of 1st Problem ----------------#
resultTest1 = spark.sql(final_query1)
resultTest1.show()
# TO DOWNLOAD FILES TO THE CONTENT FOLDER OF COLAB
#resultTest1.coalesce(1).write.mode("overwrite").csv("/content/resultTest_prb1", header=True)
# -----------------------2nd Problem mitigation ----------------#
spark.sql(final_query1).createOrReplaceTempView("Second_problem")

second_problem_query1  = """
        select winner as Player_name,
        count(*) as number_of_wins from Second_problem
        group by winner
        order by number_of_wins DESC
"""
#  -----------Sollution of 2nd Problem ----------------#
resultTest2 = spark.sql(second_problem_query1)
resultTest2.show()
# TO DOWNLOAD FILES TO THE CONTENT FOLDER OF COLAB
#resultTest2.coalesce(1).write.mode("overwrite").csv("/content/resultTest_prb2", header=True)
# -----------------------3rd Problem mitigation ----------------#

third_problem_query0 = """
                with cte_popular_move as (
                select A.eco, A.eco_name, B.game_id,
                count(*) over(partition by A.eco, B.game_id  order by A.eco) as no_of_occurances
                from eco_codesTemp A left join chess_wc_history_game_infoTemp1 B
                on A.eco = B.eco )
                select eco, eco_name ,
                count(*) over(partition by eco_name order by eco) as occurance
                from cte_popular_move
                group by eco,eco_name
                order by occurance DESC
"""

test_res6 = spark.sql(third_problem_query0).createOrReplaceTempView("third_query_test0")
#test_res6.show()

third_problem_query1 = """with cte_test_final as (
                        select eco, eco_name,
                        occurance as number_of_occurences,
                        ROW_NUMBER() over( order by occurance DESC) as MostRow,
                        ROW_NUMBER() over( order by occurance ASC) as leastRow
                        from third_query_test0 )
                        select
                        eco,
                        eco_name,
                        number_of_occurences   --,MostRow, leastRow
                        from cte_test_final
                        where  MostRow =1 or leastRow = 1
                        order by number_of_occurences DESC
                        """

#  -----------Sollution of 3rd Problem ----------------#
result_test3 = spark.sql(third_problem_query1)   #.createOrReplaceTempView("third_query_test")
result_test3.show()

# -----------------------4th Problem mitigation ----------------#

fourth_problem_query = """
                with cte_popular_move_win as (
                select A.eco, A.eco_name, B.game_id,B.winner_elo
                from eco_codesTemp A left join chess_wc_history_game_infoTemp1 B
                on A.eco = B.eco
                where B.winner_elo is not null
                )
                select eco, eco_name ,winner_elo,
                count(*) over (partition by eco order by eco) as occurance
                from cte_popular_move_win
                group by eco,eco_name,winner_elo
                order by occurance DESC
                --limit 1
"""
spark.sql(fourth_problem_query).createOrReplaceTempView("fourth_problem_query0")
#r = spark.sql("select * from fourth_problem_query0  order by occurance DEsc  limit 10")
#r.show()

fourth_problem_query1 = """
                with CTE_final AS (
                select eco, eco_name ,occurance,
                ROW_NUMBER() over(order by occurance DESC) as row_num
                from fourth_problem_query0 )
                select eco, eco_name  --, occurance, row_num
                from CTE_final
                where  row_num=1
                order by occurance DESC """

#  -----------Sollution of 4th Problem ----------------#
result_test4 = spark.sql(fourth_problem_query1 )
result_test4.show()

# -----------------------5th Problem mitigation ----------------#

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

chess_wc_history_game_infoTempTable.createOrReplaceTempView("chess_wc_history_game_infoTempTable1")
chess_wc_history_movesTempTable.createOrReplaceTempView("chess_wc_history_movesTempTable1")

#test  = spark.sql("select game_id, event ,")

# move_no , move_Sequence(even number shoub be there),game_id,event, tournament_name
#Longest and shortest game ever played in a world championship in terms of move
# Final result will have only two rows
fifth_problem_query0 = """
                        with cte_test_move as (
                        select
                        A.game_id,B.date_played,
                        A.move_no, A.move_no_pair ,
                        A.move_Sequence,
                        B.event, B.tournament_name,
                        B.winner,
                        B.loser,
                        B.winner_elo,
                        B.loser_elo,
                        ROW_NUMBER() over(partition by A.game_id, B.date_played order by A.move_Sequence DESC) as longest_move_rank
                        --ROW_NUMBER() over(partition by A.game_id, B.date_played order by A.move_Sequence ASC) as shortest_move_rank
                        from chess_wc_history_movesTempTable1 A left join chess_wc_history_game_infoTempTable1 B
                        on A.game_id =  B.game_id )
                        select game_id, date_played,
                        --move_no,
                        --longest_move_rank as move_rank,
                        move_no_pair,
                        move_Sequence, event,tournament_name,winner,loser,
                        ROW_NUMBER() over(order by move_no_pair DESC) as high_rank,
                        --ROW_NUMBER() over(order by move_no_pair ASC) as low_rank,
                        winner_elo,loser_elo
                        from cte_test_move
                        where longest_move_rank = 1 and move_no_pair%2 = 0               --in (1,2,3,4)
                         and event is not null
                        --group by game_id, date_played, move_no_pair ,move_no
                        order by move_no_pair DESC

"""
spark.sql(fifth_problem_query0).createOrReplaceTempView("fifth_problem_query1")

fifth_problem_query2 = """
                      WITH CTE_Final AS (
                      SELECT
                      game_id,
                      event,
                      tournament_name,
                      move_no_pair AS number_of_moves,
                      ROW_NUMBER() OVER ( ORDER BY move_no_pair DESC) AS longest_game_played,
                      ROW_NUMBER() OVER ( ORDER BY move_no_pair ASC) AS shortest_game_played,
                      move_sequence
                      FROM fifth_problem_query1
                      )
                    SELECT
                    game_id,
                    event,
                    tournament_name,
                    number_of_moves
                    FROM CTE_Final
                    WHERE longest_game_played = 1 OR shortest_game_played = 1
                    ORDER BY number_of_moves DESC """


#  -----------Sollution of 5th Problem ----------------#
result_test5 = spark.sql(fifth_problem_query2)
result_test5.show()

# -----------------------6th Problem mitigation ----------------#

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

sixth_problem_query0 = """
                        with cte_final_sixth as (
                        select
                        game_id,
                        event,
                        tournament_name,
                        move_no_pair as number_of_moves,
                        ROW_NUMBER()over(order by move_no_pair DESC)as high_rank_draw,
                        ROW_NUMBER()over(order by move_no_pair ASC)as low_rank_draw
                        from fifth_problem_query1
                        where winner like '%draw%' )
                        select
                        game_id,
                        event,
                        tournament_name,
                        number_of_moves
                        from cte_final_sixth
                        where high_rank_draw = 1 or low_rank_draw = 1
                        order by number_of_moves DESC"""


#  -----------Sollution of 6th Problem ----------------#
result_test6 = spark.sql(sixth_problem_query0)
result_test6.show()

# -----------------------7th Problem mitigation ----------------#

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

seventh_problem_query0 = """
                        with cte_Final_playerRanks as (
                        select
                        winner as player_name ,
                        winner_elo as elo,
                        ROW_NUMBER() over ( order by winner_elo DESC) as ranking_high,
                        ROW_NUMBER() over ( order by winner_elo ASC) as ranking_low
                        from chess_wc_history_game_infoTempTable1
                        where winner_elo is not null and loser_elo is not null )
                        select player_name, elo
                        from cte_Final_playerRanks
                        where ranking_high =1  or ranking_low = 1
                        order by elo DESC
"""


#  -----------Sollution of 7th Problem ----------------#
result_test7 = spark.sql(seventh_problem_query0)
result_test7.show()


# -----------------------8th Problem mitigation ----------------#

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

eighth_problem_query0 = """
                with cte_final_mostlossPlayer as (
                select
                game_id,
                loser as player_name,
                event,
                count(*) over(partition by loser order by event) as loss_countofPlayer
                from  chess_wc_history_game_infoTempTable1 )
                select distinct
                player_name ,
                loss_countofPlayer
                --ROW_NUMBER() over( partition by player_name, loss_countofPlayer  order by loss_countofPlayer DESC) as loss_ranking
                from cte_final_mostlossPlayer
                where player_name not like '%draw%'
                order by loss_countofPlayer DESC
 """
spark.sql(eighth_problem_query0).createOrReplaceTempView("eighth_problem_test1")


# this below query will give unique records of the highest loss the person has taken in overall
# used the offset to skip 1st 2 rows of sorted table and getting the 3rd Last Player with most Loss
eighth_problem_query1 = """
                        with cte_intermediate_lossdetails as (
                        select
                        player_name,
                        loss_countofPlayer,
                        ROW_NUMBER() over(partition by player_name order by loss_countofPlayer DESC) as row_num1
                        from eighth_problem_test1 )
                        select
                        player_name,
                        loss_countofPlayer
                        from cte_intermediate_lossdetails
                        where row_num1 = 1
                        group by player_name,loss_countofPlayer
                        order by loss_countofPlayer DESC
                        --limit 1 offset 2
"""
spark.sql(eighth_problem_query1).createOrReplaceTempView("eighth_problem_test2")

eighth_problem_query2 ="""
                        with cte_Final as (
                        select
                        player_name  ,
                        loss_countofPlayer ,
                        RANK() over( order by loss_countofPlayer DESC) as rank
                        from eighth_problem_test2 )
                        select player_name
                        from cte_Final
                        where rank = 3"""

#  -----------Sollution of 8th Problem ----------------#
result_test8 = spark.sql(eighth_problem_query2)
result_test8.show()

# -----------------------9th Problem mitigation ----------------#
# How many times players with low rating won matches with their total win Count.
# Result attributes: player_name, win_count

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()
#Required fields - game_id , event, tournament_name,winner,winner_elo,loser,loser_elo, date_played

ninth_problem_query = """
                        with cte_lowRankWinner as (
                        select
                        --A.game_id,
                        --A.date_played,
                        --A.event,
                        --A.tournament_name,
                        A.winner,A.winner_elo,
                        count(*) over(partition by A.winner order by A.winner) as countOfLowRankWinners,
                        Row_Number() over(partition by A.winner order by A.winner) as row_num,
                        B.loser,B.loser_elo
                        from chess_wc_history_game_infoTempTable1 A left join chess_wc_history_game_infoTempTable1 B
                        on A.game_id = B.game_id
                        where A.winner not like '%draw%'and A.winner_elo is not null
                        and B.loser_elo > A.winner_elo
                        group by A.winner,A.winner_elo,B.loser,B.loser_elo )
                        select
                        winner as player_name,
                        countOfLowRankWinners as win_count
                        from cte_lowRankWinner
                        where row_num = 1
                        order by countOfLowRankWinners DESC
"""


#  -----------Sollution of 9th Problem ----------------#
resultTest9 = spark.sql(ninth_problem_query)
resultTest9.show()
#---------- TO DOWNLOAD FILES TO THE CONTENT FOLDER OF COLAB ------------#
#resultTest9.coalesce(1).write.mode("overwrite").csv("/content/resultTest_prb9", header=True)





# -----------------------10th Problem mitigation ----------------#
# Move Sequence for Each Player in a Match.
# Result attributes: game_id, player_name, move_sequence, move_count

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

# REquired fields : game_id, move_no, move_no_pair, player , move , move_sequence

tenth_problem_query = """
                        with cte_movSequence as (
                        select
                        game_id, move_no,
                        move_no_pair, player , notation,
                        move , move_sequence,
                        ROW_NUMBER()over(partition by game_id,player order by move_no_pair DESC) as movRowSeq
                        from chess_wc_history_movesTemp )
                        select
                        game_id,
                        player as player_name ,
                        move_sequence,
                        move_no as move_count
                        --(LENGTH(move_sequence) - LENGTH(REPLACE(move_sequence, '|', '')) + 1) AS move_count,
                        --movRowSeq
                        from cte_movSequence
                        where movRowSeq = 1
                        order by  move_no DESC """


#  -----------Sollution of 10th Problem ----------------#
result_test10 = spark.sql(tenth_problem_query)
result_test10.show()


# -----------------------11th Problem mitigation ----------------#

#Total Number of games where losing player has more Captured score than Winning player.
#Hint: Captured score is cumulative, i.e., for 3rd capture it will have score for 1, 2, and 3rd.
#Result attributes: total_number_of_games Final result will have only one row

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()


# REquired fields :
# from chess_wc_history_movesTemp - captured_score_for_white,captured_score_for_black , game_id,
# from chess_wc_history_game_infoTempTable1 : winner, winner_elo, loser,loser_elo ,game_id

eleventh_problem_query0 = """
                        select
                        distinct
                        game_id,
                        sum(captured_score_for_white) over(partition by game_id order by game_id) as CumilativeOfWhite,
                        sum(captured_score_for_black) over(partition by game_id order by game_id) as CumilativeOfBlack
                        from chess_wc_history_movesTemp
                        where captured_score_for_white != 0 and captured_score_for_black !=0

"""
spark.sql(eleventh_problem_query0).createOrReplaceTempView("eleventh_query1")

eleventh_problem_query1 = """
                        select
                        A.game_id,
                        A.CumilativeOfWhite,
                        A.CumilativeOfBlack,
                        B.white, B.black,
                        B.winner,B.loser
                        from  eleventh_query1 A left join  chess_wc_history_game_infoTempTable1 B
                        on A.game_id = B.game_id
                        where B.winner not like '%draw%' and B.winner_elo is not null

"""
spark.sql(eleventh_problem_query1).createOrReplaceTempView("eleventh_query2")
eleventh_problem_query2 = """
                        with cte_finalGameScore as (
                        select
                        game_id ,
                        case
                        when winner == black then CumilativeOfBlack
                                else CumilativeOfWhite
                        end as winnerScore,
                        case
                        when loser == black then CumilativeOfBlack
                                else CumilativeOfWhite
                        end as loserScore,
                        CumilativeOfBlack,
                        CumilativeOfWhite,
                        white,black,
                        winner, loser
                        from eleventh_query2 )
                        select
                        count(*) as total_number_of_games
                        from cte_finalGameScore
                        where loserScore > winnerScore

"""
#  -----------Sollution of 11th Problem ----------------#
result_test11 = spark.sql(eleventh_problem_query2)
result_test11.show()



# -----------------------12th Problem mitigation ----------------#

#List All Perfect Tournament with Winner Name.
#Chess Funda: Perfect Tournament means a player has won all the matches excluding draw matches. e.g Player A has won 5 matches out of 7
#Matches in tournament where 2 matches are draw and player B has won 0 matches)
#Result attributes: winner_name, tournament_name

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

# to find the perfect tournament where the winner not would have came in loser list even at once
# here we can use the 1st problems solutions answer to answer this question
spark.sql(final_query1).createOrReplaceTempView("WinnerOfAllTournaments")

spark.sql("select tournament_name,loser from chess_wc_history_game_infoTempTable1  where tournament_name like 'WorldChamp%' group by tournament_name,loser order by tournament_name").createOrReplaceTempView("LoserOfTournaments")

twelth_problem_query0 = """
                with CTE_PerfectMatchChecking as (
                select
                A.winner,
                A.tournament_name,
                B.loser,
                (case
                when (A.winner = B.loser and A.tournament_name = B.tournament_name)
                then 1
                else 0
                end )as ValueWinLose
                from WinnerOfAllTournaments A left join  LoserOfTournaments B
                on A.tournament_name = B.tournament_name
                where  --A.tournament_name like '%WorldChamp1889%' and
                 loser not like '%draw%' )
                select  distinct
                winner,
                tournament_name,
                sum(ValueWinLose)over(partition by tournament_name order by tournament_name) as WinnerLosingValue
                from  CTE_PerfectMatchChecking
                order by tournament_name DESC """

spark.sql(twelth_problem_query0).createOrReplaceTempView("twelth_query1")
#  -----------Sollution of 12th Problem ----------------#

result_test12 = spark.sql("select winner,tournament_name as Perferct_tournament_name from twelth_query1 where WinnerLosingValue = 0 ")
result_test12.show()




# -----------------------13th Problem mitigation ----------------#
# Player with highest winning ratio.
# Hint: Winning ratio: (Number of rounds won)/(Number of rounds played)
# Result attributes: player_name
# Final result will have only one row


# chess_wc_history_movesTempTable.show()
# eco_codesTempTable.show()
# chess_wc_history_game_infoTempTable.show()


# REquired fields :

thirteen_problem_query0 = """
       with CTE_FINAL as (
       select
       white, black,
       winner,
       count(*) over(partition by tournament_name, winner order by tournament_name) as winCount ,
       max(round) over(partition by  tournament_name order by tournament_name) as NumRoundPlayed,
       tournament_name,
       round,event
       from chess_wc_history_game_infoTemp1
       --where winner not like '%draw%'
       order by tournament_name,round DESC )
       select
       Distinct
       winner as player_name,
       round((winCount/NumRoundPlayed),2) as winningRatio
       from
       CTE_FINAL
       where winner not like '%draw%'
       order by winningRatio DESC
       limit 1
       """

#  -----------Sollution of 13th Problem ---------------- #

result_test13 = spark.sql(thirteen_problem_query0)
result_test13.show()



# -----------------------14th Problem mitigation ----------------#
# Player who had given checkmate with Pawn.
# Note: Consider all events for this query
# Result attributes: player_name
# Final result will have only one row
# P for pawn


#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

fourteenth_problem_query0 = """
                        with CTE_CheckmateWithPawn as (
                        select
                        A.game_id ,
                        B.winner,
                        B.loser,
                        B.winner_elo,
                        A.move, A.player,
                        A.white_pawn_count,
                        A.black_pawn_count,
                        A.piece,
                        A.is_check_mate
                        from chess_wc_history_movesTemp  A left join  chess_wc_history_game_infoTemp B
                        on A.game_id = B.game_id
                        where  A.is_check_mate != 0 )
                        select
                        winner as player_name
                        from CTE_CheckmateWithPawn
                        where piece like '%P%' """

#  -----------Sollution of 14th Problem ----------------#
result_test14 = spark.sql(fourteenth_problem_query0)
result_test14.show()



# -----------------------15th Problem mitigation ----------------#
# List games where player has won game without queen.
# Result attributes: game_id, event, player_name


#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

fifteenth_problem_query0 = """
                        with cte_testNoQueen as (
                        select
                        game_id ,
                        player,
                        move_no,
                        move_no_pair,
                        case when piece like '%Q%' then 1 else 0
                        end as QueenUsage
                        from chess_wc_history_movesTemp  )
                        --group by game_id, player
                        select  distinct
                        game_id ,
                        player,
                        sum(QueenUsage) over (partition by game_id, player order by game_id) as QueenUsageVal
                        from cte_testNoQueen
"""
spark.sql(fifteenth_problem_query0).createOrReplaceTempView("fifteenth_query1")

fifteenth_problem_query1 = """
                with CTE_winWithoutQueen as (
                SELECT
                A.game_id,
                A.player,
                B.winner as Winner_player_name,
                B.event,
                A.QueenUsageVal
                from
                fifteenth_query1 A left join  chess_wc_history_game_infoTemp B
                on A.game_id = B.game_id and A.player = B.winner
                where  A.QueenUsageVal = 0
                and B.winner not like '%draw%' )
                select
                game_id, event,
                Winner_player_name as player_name
                from CTE_winWithoutQueen
                order by event DESC """


#  -----------Sollution of 15th Problem ----------------#
result_test15 = spark.sql(fifteenth_problem_query1)
result_test15.show()
#result_test15.coalesce(1).write.mode("overwrite").csv("/content/resultTest_prb15", header=True)

# list of the sparkDataframes(all that are final results )so that
# we can use this to pass it function and convert that in pandas DF


spark_df_list = [
resultTest1,
resultTest2,
result_test3,
result_test4,
result_test5,
result_test6,
result_test7,
result_test8,
resultTest9,
result_test10,
result_test11,
result_test12,
result_test13,
result_test14,
result_test15]


# Google Drive path
drive_path = 'DE_SOLUTION_Gajendra_Ahirkar/results/'


# Using the function to save dataframes to given path in csv format
spark_dataframes_to_csv(spark_df_list,drive_path)



+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|           Carlsen,M| WorldChamp2021|
|     Carlsen, Magnus| WorldChamp2016|
|     Carlsen, Magnus| WorldChamp2014|
|     Carlsen, Magnus| WorldChamp2013|
|           Gelfand,B| WorldChamp2012|
|             Anand,V| WorldChamp2010|
|             Anand,V| WorldChamp2008|
|             Anand,V| WorldChamp2007|
|           Kramnik,V| WorldChamp2006|
|              Leko,P| WorldChamp2004|
|           Kramnik,V| WorldChamp2000|
|      Kasparov, Gary| WorldChamp1990|
|      Kasparov, Gary| WorldChamp1987|
|      Kasparov, Gary| WorldChamp1986|
|      Kasparov, Gary| WorldChamp1985|
|     Karpov, Anatoly| WorldChamp1984|
|     Karpov, Anatoly| WorldChamp1981|
|     Karpov, Anatoly| WorldChamp1978|
|Fischer, Robert J...| WorldChamp1972|
|    Spassky, Boris V| WorldChamp1969|
+--------------------+---------------+
only showing top 20 rows

+--------------------+--------------+
